In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('result.csv')
df.head()

,cycle,ambient_temperature,datetime,capacity,voltage_measured,current_measured,temperature_measured,current_load,voltage_load,time,soh_calculated
0,1,24,2008-04-02 15:25:41,1.856487,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000,1.0
1,1,24,2008-04-02 15:25:41,1.856487,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781,1.0
2,1,24,2008-04-02 15:25:41,1.856487,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703,1.0
3,1,24,2008-04-02 15:25:41,1.856487,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781,1.0
4,1,24,2008-04-02 15:25:41,1.856487,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922,1.0


In [3]:
df.drop(['current_load', 'voltage_load', 'cycle', 'ambient_temperature',
         'datetime', 'capacity' ], axis=1, inplace= True)
df.describe

<bound method NDFrame.describe of         voltage_measured  current_measured  temperature_measured      time  \
0               4.191492         -0.004902             24.330034     0.000   
1               4.190749         -0.001478             24.325993    16.781   
2               3.974871         -2.012528             24.389085    35.703   
3               3.951717         -2.013979             24.544752    53.781   
4               3.934352         -2.011144             24.731385    71.922   
...                  ...               ...                   ...       ...   
185716          3.443760         -0.002426             35.383979  2686.359   
185717          3.453271         -0.000981             35.179732  2700.546   
185718          3.461963          0.000209             34.977000  2714.640   
185719          3.469907          0.001516             34.785943  2728.750   
185720          3.477277         -0.001940             34.581660  2742.843   

        soh_calculated  
0   

In [4]:
input_para = [ 'voltage_measured', 'current_measured' ]
train_dataset = df[input_para]

print(train_dataset.shape)

(185721, 2)


In [5]:
output_var = df['temperature_measured'] 
print(output_var.shape)

(185721,)


In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,  y_train, y_test = train_test_split(train_dataset, output_var, test_size= 0.20)

In [7]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(148576, 2)
(37145, 2)
(148576,)
(37145,)


In [8]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=30)

In [9]:
history = model.fit(x_train, y_train)

In [10]:
temp_pred = model.predict(x_test)
print(temp_pred.shape)
print(temp_pred)

(37145,)
[35.02444147 39.35335307 33.25652269 ... 35.98772963 28.33980838
 36.25811498]


In [11]:
y_test

179143    34.353655
4729      36.824469
34610     34.286237
30459     32.899792
113543    24.446878
            ...    
53573     29.728661
87351     32.805687
70346     35.549533
94301     27.302274
48399     36.611603
Name: temperature_measured, Length: 37145, dtype: float64

In [12]:
df_pred = pd.DataFrame( temp_pred , y_test)
df_pred.reset_index(inplace=True)

In [13]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37145 entries, 0 to 37144
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   temperature_measured  37145 non-null  float64
 1   0                     37145 non-null  float64
dtypes: float64(2)
memory usage: 580.5 KB


In [14]:
df_pred = df_pred.rename(columns={0: 'temp_predicted'})

In [15]:
df_pred.head()

,temperature_measured,temp_predicted
0,34.353655,35.024441
1,36.824469,39.353353
2,34.286237,33.256523
3,32.899792,32.319793
4,24.446878,24.479903


In [16]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score
print('R2 score:', r2_score(y_test, temp_pred))
print('MAE:', mean_absolute_error(y_test, temp_pred))
print('MSE:', mean_squared_error(y_test, temp_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, temp_pred)))

R2 score: 0.9342758818761713
MAE: 0.7835154274864046
MSE: 1.0712731702658866
RMSE: 1.035023270398249


In [17]:
# df_sorted = df_pred.sort_values('temperature_measured', ascending=True)
# df_sorted.reset_index(inplace= True)
# df_sorted['Index'] = df_sorted.index
# df_sorted.head(10)

In [18]:
# plt.figure(figsize = (15,8))
# plt.plot(df_sorted["Index"], df_sorted['temperature_measured'], 'r' ,label='Actual Temperature')
# plt.scatter(df_sorted["Index"], df_sorted['temp_predicted'], s=2 ,alpha=0.5, label='Predicted by RF')
# plt.ylabel('Temperature')
# plt.xlabel('Tested Instances')
# plt.title('Temperature Estimation Comparative Analysis')
# plt.legend()

In [19]:
import pickle
pickle_out = open('temp_model.pkl', 'wb')
pickle.dump(model, pickle_out)
pickle_out.close()

In [20]:
model = pickle.load(open('model.pkl', 'rb'))

In [21]:
model.predict(x_test)

array([28.40596617, 28.45976479, 38.02181924, ..., 31.41327617,
       26.63286526, 26.13823471])

In [22]:
model.predict([[3.754, -2.111 ]])

array([24.39724012])

In [1]:
import pandas as pd
data = pd.read_excel('test.xlsx')
data.head()

,voltage_measured,current_measured
0,4.191492,-0.004902
1,4.190749,-0.001478
2,3.974871,-2.012528
3,3.951717,-2.013979
4,3.934352,-2.011144


In [2]:
data.columns

Index(['voltage_measured', 'current_measured'], dtype='object')